In [1]:
import os
import time
import subprocess
import tkinter as tk
from tkinter import scrolledtext
from pathlib import Path

def get_pipewire_permissions():
    try:
        result = subprocess.run(["pw-cli", "permissions"], capture_output=True, text=True)
        return result.stdout
    except FileNotFoundError:
        return "PipeWire CLI not found. Install pipewire-tools."

def get_pipewire_owners():
    try:
        result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
        processes = [line for line in result.stdout.split('\n') if "pipewire" in line]
        return "\n".join(processes)
    except Exception as e:
        return str(e)

def get_pipewire_configurations():
    config_paths = ["/etc/pipewire", Path.home() / ".config/pipewire"]
    config_files = []
    for path in config_paths:
        if os.path.exists(path):
            for root, _, files in os.walk(path):
                for file in files:
                    config_files.append(os.path.join(root, file))
    return "\n".join(config_files)

def get_pipewire_modules():
    try:
        result = subprocess.run(["pw-cli", "info", "all"], capture_output=True, text=True)
        return result.stdout
    except FileNotFoundError:
        return "PipeWire CLI not found. Install pipewire-tools."

def update_gui():
    text_area.delete("1.0", tk.END)
    text_area.insert(tk.END, "=== PipeWire Monitoring Log ===\n")
    text_area.insert(tk.END, f"Permissions:\n{get_pipewire_permissions()}\n")
    text_area.insert(tk.END, f"Owners:\n{get_pipewire_owners()}\n")
    text_area.insert(tk.END, f"Configurations:\n{get_pipewire_configurations()}\n")
    text_area.insert(tk.END, f"Modules:\n{get_pipewire_modules()}\n")
    text_area.insert(tk.END, "================================\n")
    root.after(60000, update_gui)  # Update every 60 seconds

root = tk.Tk()
root.title("PipeWire Monitor")

text_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=30)
text_area.pack(padx=10, pady=10)

update_gui()
root.mainloop()
